In [1]:
import subprocess
from time import sleep
from datetime import date, datetime
import random
import logging

import argparse
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l1

from tensorflow.keras.models import Model
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper

import visualkeras
import hls4ml

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, WindowFunctions, DetrendOperations, NoiseTypes
from collections import Counter
from sklearn.preprocessing import RobustScaler

2021-11-14 21:40:59.513520: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/home/nexuscore/anaconda3/envs/hls4ml-pynq-tutorial/lib/python3.7/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [8]:
import hls4ml

## Classes: 

#### Zene: 0

#### Meditáció: 1

#### Figyelem: 2

###### Az osztályinformáció szokásos módon a -4. oszlopban található a .txt reprezentációban, és -4. sor itt

In [43]:
tudatallapot_full = DataFilter.read_file('./your_filename.txt')

In [44]:
tudatallapot_full.shape

(32, 1687552)

In [45]:
y_all = tudatallapot_full[-4, :]

In [46]:
y_all.shape

(1687552,)

In [ ]:
Ezen cellával lehetséges ellenőrizni az adatok eloszlását

In [47]:
music = 0
meditation = 0
attention = 0

for i in range(y_all.shape[0]):
    if(y_all[i] == 0.0):
        music += 1
    if(y_all[i] == 1.0):
        meditation += 1
    if(y_all[i] == 2.0):
        attention += 1

print(f"arányok különböző esetekben:\n music: {music / y_all.shape[0] * 100} \n meditation: {meditation / y_all.shape[0] * 100} \n attention: {attention / y_all.shape[0] * 100} \n")

arányok különböző esetekben:
 music: 33.34344660194174 
 meditation: 33.34344660194174 
 attention: 33.31310679611651 



Az alábbi cellával lehetséges az adatok előfeldolgozását elvégezni. A feldolgozatlan adatokat a tudatallapot_full változó tárolja

In [48]:
for i in range(1, 17):
    for z in range(int(tudatallapot_full.shape[1] / 512)):
        # print(f'tudatallapot_full.shape : {tudatallapot_full[i].shape}')
        DataFilter.detrend(tudatallapot_full[i][0 + z * 512 : 512 + z * 512], DetrendOperations.CONSTANT.value)

for i in range(1, 17):
    for z in range(int(tudatallapot_full.shape[1] / 512)):
        # print(f'tudatallapot_full.shape : {tudatallapot_full[i].shape}')
        DataFilter.remove_environmental_noise(tudatallapot_full[i][0 + z * 512 : 512 + z * 512], 125, NoiseTypes.FIFTY.value)
        DataFilter.perform_wavelet_denoising(tudatallapot_full[i][0 + z * 512 : 512 + z * 512], 'haar', 2)
        DataFilter.perform_highpass(tudatallapot_full[i][0 + z * 512 : 512 + z * 512], 125, 3.0, 4, FilterTypes.BUTTERWORTH.value, 0)
        tudatallapot_full[i][0 + z * 512 : 512 + z * 512] = RobustScaler(quantile_range=(25, 75), copy="False").fit_transform((tudatallapot_full[i][0 + z * 512 : 512 + z * 512]).reshape(-1, 1)).reshape(-1)
        
X_full = tudatallapot_full[1:17]

X_all = np.transpose(X_full)

X_train = np.resize(X_all, (int(X_all.shape[0] / 512), 512, 16, 1))

print(f'X_train.shape : {X_train.shape}')

y_all = tudatallapot_full[-4, :]

ids = np.random.permutation(X_train.shape[0])

X_shuffled = X_train[ids]

try:
    y = np.zeros(int(int(y_all.shape[0]) / 512))

    outer = 0

    for i in range(int(y_all.shape[0])):
        y[i] = y_all[outer]
        outer += 512
        print(outer)
except:
    pass

y_shuffled = y[ids]

X_train.shape : (3296, 512, 16, 1)
512
1024
1536
2048
2560
3072
3584
4096
4608
5120
5632
6144
6656
7168
7680
8192
8704
9216
9728
10240
10752
11264
11776
12288
12800
13312
13824
14336
14848
15360
15872
16384
16896
17408
17920
18432
18944
19456
19968
20480
20992
21504
22016
22528
23040
23552
24064
24576
25088
25600
26112
26624
27136
27648
28160
28672
29184
29696
30208
30720
31232
31744
32256
32768
33280
33792
34304
34816
35328
35840
36352
36864
37376
37888
38400
38912
39424
39936
40448
40960
41472
41984
42496
43008
43520
44032
44544
45056
45568
46080
46592
47104
47616
48128
48640
49152
49664
50176
50688
51200
51712
52224
52736
53248
53760
54272
54784
55296
55808
56320
56832
57344
57856
58368
58880
59392
59904
60416
60928
61440
61952
62464
62976
63488
64000
64512
65024
65536
66048
66560
67072
67584
68096
68608
69120
69632
70144
70656
71168
71680
72192
72704
73216
73728
74240
74752
75264
75776
76288
76800
77312
77824
78336
78848
79360
79872
80384
80896
81408
81920
82432
82944
83456
83968
8

In [49]:
model = Sequential()

model.add(Input(shape=X_train.shape[1:]))

model.add(QConv2DBatchnorm(8, kernel_size=(3,3), strides=(1,1), padding = "same",
                         kernel_quantizer="quantized_bits(6,0,alpha=1)", 
                         bias_quantizer="quantized_bits(6,0,alpha=1)",
                         kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                         ))
model.add(QActivation('quantized_relu(6)'))

model.add(MaxPooling2D(pool_size = (2,2))) 

model.add(QConv2DBatchnorm(10, kernel_size=(3,3), strides=(1,1), padding = "same",
                         kernel_quantizer="quantized_bits(6,0,alpha=1)", 
                         bias_quantizer="quantized_bits(6,0,alpha=1)",
                         kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                         ))
model.add(QActivation('quantized_relu(6)'))

model.add(MaxPooling2D(pool_size = (2,2))) 

model.add(QConv2DBatchnorm(10, kernel_size=(3,3), strides=(1,1), padding = "same",
                         kernel_quantizer="quantized_bits(6,0,alpha=1)", 
                         bias_quantizer="quantized_bits(6,0,alpha=1)",
                         kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                         ))
model.add(QActivation('quantized_relu(6)'))

# model.add(MaxPooling2D(pool_size = (2,2))) 

model.add(QConv2DBatchnorm(12, kernel_size=(3,3), strides=(1,1), padding = "same",
                         kernel_quantizer="quantized_bits(6,0,alpha=1)", 
                         bias_quantizer="quantized_bits(6,0,alpha=1)",
                         kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                         ))
model.add(QActivation('quantized_relu(6)'))

model.add(MaxPooling2D(pool_size = (4,4)))

model.add(Flatten())

model.add(QDense(5,
            kernel_quantizer="quantized_bits(6,0,alpha=1)",
            kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False))
model.add(BatchNormalization())
model.add(QActivation('quantized_relu(6)'))

model.add(QDense(16,
            kernel_quantizer="quantized_bits(6,0,alpha=1)",
            kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False))
model.add(BatchNormalization())
model.add(QActivation('quantized_relu(6)'))

model.add(Dense(3))
model.add(Activation('softmax'))

print(model.summary())

for layer in model.layers:
    if layer.__class__.__name__ in ['Conv2D', 'Dense', 'QDense', 'QConv2DBatchnorm']:
        w = layer.get_weights()[0]
        layersize = np.prod(w.shape)
        print("{}: {}".format(layer.name,layersize)) # 0 = weights, 1 = biases
        if (layersize > 4096): # assuming that shape[0] is batch, i.e., 'None'
           print("Layer {} is too large ({}), are you sure you want to train?".format(layer.name,layersize))

2021-11-14 09:48:42.580761: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-14 09:48:42.619759: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-14 09:48:42.619784: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nexyys): /proc/driver/nvidia/version does not exist
2021-11-14 09:48:42.620202: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-14 09:48:42.656246: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3792555000 Hz
2021-11-14 09:48:42.656812: I tensorflow/co

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
q_conv2d_batchnorm (QConv2DB (None, 512, 16, 8)        113       
_________________________________________________________________
q_activation (QActivation)   (None, 512, 16, 8)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 8, 8)         0         
_________________________________________________________________
q_conv2d_batchnorm_1 (QConv2 (None, 256, 8, 10)        771       
_________________________________________________________________
q_activation_1 (QActivation) (None, 256, 8, 10)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 4, 10)        0         
_________________________________________________________________
q_conv2d_batchnorm_2 (QConv2 (None, 128, 4, 10)        9

In [50]:
# Print the quantized layers
from qkeras.autoqkeras.utils import print_qmodel_summary
print_qmodel_summary(model)        

q_conv2d_batchnorm   f=8 quantized_bits(6,0,0,alpha=1) quantized_bits(6,0,0,alpha=1) 
q_activation         quantized_relu(6)
q_conv2d_batchnorm_1 f=10 quantized_bits(6,0,0,alpha=1) quantized_bits(6,0,0,alpha=1) 
q_activation_1       quantized_relu(6)
q_conv2d_batchnorm_2 f=10 quantized_bits(6,0,0,alpha=1) quantized_bits(6,0,0,alpha=1) 
q_activation_2       quantized_relu(6)
q_conv2d_batchnorm_3 f=12 quantized_bits(6,0,0,alpha=1) quantized_bits(6,0,0,alpha=1) 
q_activation_3       quantized_relu(6)
q_dense              u=5 quantized_bits(6,0,0,alpha=1) 
batch_normalization_4 is normal keras bn layer
q_activation_4       quantized_relu(6)
q_dense_1            u=16 quantized_bits(6,0,0,alpha=1) 
batch_normalization_5 is normal keras bn layer
q_activation_5       quantized_relu(6)



In [51]:
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow as tf
from tensorflow import keras

batch_size = 32
train_size = int(int(y_all.shape[0]) / 512)

NSTEPS = int(train_size*0.9)  // batch_size #90% train, 10% validation in 10-fold cross validation
print('Number of training steps per epoch is {}'.format(NSTEPS))

# Prune all convolutional and dense layers gradually from 0 to 50% sparsity every 2 epochs, 
# ending by the 10th epoch
def pruneFunction(layer):
    pruning_params = {'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity = 0.0,
                                                                   final_sparsity = 0.50, 
                                                                   begin_step = NSTEPS*2, 
                                                                   end_step = NSTEPS*10, 
                                                                   frequency = NSTEPS)
                     }
    if isinstance(layer, tf.keras.layers.Conv2D):
      return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    if isinstance(layer, tf.keras.layers.Dense) and layer.name!='output_dense':
      return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)  
    return layer

qmodel_pruned = keras.models.clone_model(model, clone_function=pruneFunction)

Number of training steps per epoch is 92
Instructions for updating:
Please use `layer.add_weight` method instead.


In [52]:
train = True

n_epochs = 300
if train:
    LOSS        = tf.keras.losses.SparseCategoricalCrossentropy()
    OPTIMIZER   = tf.keras.optimizers.Adam(learning_rate=3E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True) 
    qmodel_pruned.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])

    callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=210, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=15, verbose=1),
            pruning_callbacks.UpdatePruningStep()
            ]  

    start = time.time()
    history = qmodel_pruned.fit(X_shuffled, y_shuffled,
                          epochs = n_epochs,
                          callbacks = callbacks, 
                          verbose=1,
                          validation_split=0.2)     
    end = time.time()
    print('\n It took {} minutes to train!\n'.format( (end - start)/60.))

    qmodel_pruned.save('TUDATALLAPOT_FULL_QUANTIZED_VER0_312.h5')

else:
    from qkeras.utils import _add_supported_quantized_objects
    from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
    
    co = {}
    _add_supported_quantized_objects(co)
    co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude
    qmodel_pruned = tf.keras.models.load_model('TUDATALLAPOT_FULL_QUANTIZED_VER0_312.h5', custom_objects=co)

Epoch 1/300
83/83 [==============================] - 6s 76ms/step - loss: 1.0354 - accuracy: 0.4882 - val_loss: 1.1494 - val_accuracy: 0.3348
Epoch 2/300
83/83 [==============================] - 6s 73ms/step - loss: 0.8469 - accuracy: 0.6153 - val_loss: 0.9589 - val_accuracy: 0.5576
Epoch 3/300
83/83 [==============================] - 6s 71ms/step - loss: 0.7793 - accuracy: 0.6612 - val_loss: 1.0402 - val_accuracy: 0.5045
Epoch 4/300
83/83 [==============================] - 6s 72ms/step - loss: 0.7461 - accuracy: 0.6734 - val_loss: 1.3314 - val_accuracy: 0.3924
Epoch 5/300
83/83 [==============================] - 6s 72ms/step - loss: 0.7314 - accuracy: 0.6958 - val_loss: 1.1346 - val_accuracy: 0.4848
Epoch 6/300
83/83 [==============================] - 6s 71ms/step - loss: 0.6869 - accuracy: 0.7083 - val_loss: 1.0893 - val_accuracy: 0.4424
Epoch 7/300
83/83 [==============================] - 6s 73ms/step - loss: 0.6606 - accuracy: 0.7215 - val_loss: 0.7293 - val_accuracy: 0.6712
Epoch 

Epoch 111/300
83/83 [==============================] - 6s 70ms/step - loss: 0.1404 - accuracy: 0.9659 - val_loss: 0.5224 - val_accuracy: 0.8348
Epoch 112/300
83/83 [==============================] - 6s 70ms/step - loss: 0.1326 - accuracy: 0.9674 - val_loss: 0.5575 - val_accuracy: 0.8288
Epoch 113/300
83/83 [==============================] - 6s 70ms/step - loss: 0.1362 - accuracy: 0.9681 - val_loss: 0.4610 - val_accuracy: 0.8576
Epoch 114/300
83/83 [==============================] - 6s 73ms/step - loss: 0.1260 - accuracy: 0.9727 - val_loss: 0.4654 - val_accuracy: 0.8636
Epoch 115/300
83/83 [==============================] - 6s 71ms/step - loss: 0.1476 - accuracy: 0.9605 - val_loss: 0.4563 - val_accuracy: 0.8667
Epoch 116/300
83/83 [==============================] - 6s 71ms/step - loss: 0.1373 - accuracy: 0.9651 - val_loss: 0.4740 - val_accuracy: 0.8682
Epoch 117/300
83/83 [==============================] - 6s 71ms/step - loss: 0.1361 - accuracy: 0.9708 - val_loss: 0.4924 - val_accuracy:

83/83 [==============================] - 6s 70ms/step - loss: 0.1194 - accuracy: 0.9769 - val_loss: 0.4524 - val_accuracy: 0.8652
Epoch 217/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1291 - accuracy: 0.9700 - val_loss: 0.4607 - val_accuracy: 0.8667
Epoch 218/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1315 - accuracy: 0.9674 - val_loss: 0.4715 - val_accuracy: 0.8606
Epoch 219/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1390 - accuracy: 0.9636 - val_loss: 0.5626 - val_accuracy: 0.8152
Epoch 220/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1361 - accuracy: 0.9666 - val_loss: 0.5022 - val_accuracy: 0.8455
Epoch 221/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1332 - accuracy: 0.9681 - val_loss: 0.5766 - val_accuracy: 0.8076
Epoch 222/300
83/83 [==============================] - 6s 69ms/step - loss: 0.1282 - accuracy: 0.9708 - val_loss: 0.5158 - val_accuracy: 0.8424
Epoch 

## Saját adatok biztosítása nélkül ezen alábbi cellától lehetséges a flow-t követni.

In [2]:
from qkeras.utils import _add_supported_quantized_objects
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
import tensorflow as tf
model = tf.keras.models.load_model('TUDATALLAPOT_FULL_QUANTIZED_VER0_312.h5',custom_objects={'PruneLowMagnitude': pruning_wrapper.PruneLowMagnitude,
                                                                         'QDense': QDense, 
                                                                         'QConv2DBatchnorm': QConv2DBatchnorm, 
                                                                         'QActivation': QActivation})
model  = strip_pruning(model)

Instructions for updating:
Please use `layer.add_weight` method instead.


2021-11-14 22:03:23.693074: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-14 22:03:23.729760: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-14 22:03:23.729772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nexyys): /proc/driver/nvidia/version does not exist
2021-11-14 22:03:23.729986: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-14 22:03:23.765439: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3793050000 Hz
2021-11-14 22:03:23.765995: I tensorflow/co

### Az alábbi művelet futtatása időigényes lehet számítógépspecifikációtól függően

#### Az output dir-ben specifikált TUDATALLAPOT_QUANTIZED_VER0_312_FULL mappában lehetséges megtalálni a kimeneti vivado projektet, és az egyéb járulékos fileokat.
#### Ezen időigényees feladat egyszerűsítése érdekében már megtalálható a repo gyökerében ezen mappa, az alábbi cellában a  hls_model paraméterezésénél az output_dir paraméternél azért szerepel TUDATALLAPOT_QUANTIZED_VER0_312_FULL_V2 hogy ne legyen konfliktus az általam biztosított és a generált verzió között verzió között

In [56]:
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

hls_config['Model'] = {}
hls_config['Model']['ReuseFactor'] = 2
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['Precision'] = 'ap_fixed<14,6>'
hls_config['LayerName']['q_conv2d_batchnorm']['ReuseFactor'] = 16
hls_config['LayerName']['q_conv2d_batchnorm_1']['ReuseFactor'] = 16
hls_config['LayerName']['q_conv2d_batchnorm_2']['ReuseFactor'] = 16
hls_config['LayerName']['q_conv2d_batchnorm_3']['ReuseFactor'] = 16
hls_config['LayerName']['q_dense']['ReuseFactor'] = 16
hls_config['LayerName']['q_dense_1']['ReuseFactor'] = 16
hls_config['LayerName']['dense']['ReuseFactor'] = 16

hls_model = hls4ml.converters.convert_from_keras_model(model=model,
                                                       output_dir="TUDATALLAPOT_QUANTIZED_VER0_312_FULL_V2",
                                                       part='xc7z020clg400-1',
                                                       backend='VivadoAccelerator',
                                                       io_type='io_stream', 
                                                       hls_config=hls_config, 
                                                       )

hls_model.build(csim=False, synth=True, export=True)
hls4ml.report.read_vivado_report('TUDATALLAPOT_QUANTIZED_VER0_312_FULL_V2/')

hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)

Interpreting Sequential
Topology:
Layer name: input_1, layer type: Input
Layer name: q_conv2d_batchnorm, layer type: QConv2DBatchnorm
Layer name: q_activation, layer type: QActivation
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: q_conv2d_batchnorm_1, layer type: QConv2DBatchnorm
Layer name: q_activation_1, layer type: QActivation
Layer name: max_pooling2d_1, layer type: MaxPooling2D
Layer name: q_conv2d_batchnorm_2, layer type: QConv2DBatchnorm
Layer name: q_activation_2, layer type: QActivation
Layer name: q_conv2d_batchnorm_3, layer type: QConv2DBatchnorm
Layer name: q_activation_3, layer type: QActivation
Layer name: max_pooling2d_2, layer type: MaxPooling2D
Layer name: q_dense, layer type: QDense
Layer name: batch_normalization_4, layer type: BatchNormalization
Layer name: q_activation_4, layer type: QActivation
Layer name: q_dense_1, layer type: QDense
Layer name: batch_normalization_5, layer type: BatchNormalization
Layer name: q_activation_5, layer type: QActi

INFO: [HLS 200-489] Unrolling loop 'LineBufferShift' (firmware/nnet_utils/nnet_conv_stream.h:228) in function 'void nnet::shift_line_buffer<nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 10u>, config9>(FORWARD_REFERENCE const&, ap_shift_reg<FORWARD_REFERENCE::value_type, FORWARD_REFERENCE::in_width> (*) [FORWARD_REFERENCE::n_chan], FORWARD_REFERENCE::value_type*)' completely with a factor of 1.
INFO: [HLS 200-489] Unrolling loop 'LineBufferShift' (firmware/nnet_utils/nnet_conv_stream.h:228) in function 'void nnet::shift_line_buffer<nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 8u>, config5>(FORWARD_REFERENCE const&, ap_shift_reg<FORWARD_REFERENCE::value_type, FORWARD_REFERENCE::in_width> (*) [FORWARD_REFERENCE::n_chan], FORWARD_REFERENCE::value_type*)' completely with a factor of 1.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_types.h:32) in function 'nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>::operator=(nnet::array

INFO: [XFORM 203-603] Inlining function 'nnet::pooling2d_buffer_cl<nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 10u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, config9>' into 'nnet::pooling2d_cl<nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 10u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, config9>' (firmware/nnet_utils/nnet_pooling_stream.h:251).
INFO: [XFORM 203-603] Inlining function 'nnet::fill_zero<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, config31>' into 'nnet::zeropad2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, config31>' (firmware/nnet_utils/nnet_padding_stream.h:75).
INFO: [XFORM 203-603] Inlining function 'nnet::fill_zero<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, config31>' into 'nnet::zeropad2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 10u>, nn

INFO: [XFORM 203-603] Inlining function 'nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>::operator[]' into 'nnet::relu<nnet::array<ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 5u>, nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>, relu_config21>' (firmware/nnet_utils/nnet_activation_stream.h:70).
INFO: [XFORM 203-603] Inlining function 'nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>::operator[]' into 'nnet::relu<nnet::array<ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 5u>, nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>, relu_config21>' (firmware/nnet_utils/nnet_activation_stream.h:69).
INFO: [XFORM 203-603] Inlining function 'nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>::operator[].1' into 'nnet::array<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)0, 0>, 5u>::operator=' (firmware/nnet_utils/nnet_types.h:34).
INFO: [XFORM 203-603] Inlining function 'nnet::product::mult<ap_fixed<6, 1, (ap_q_mode)0, (ap_o_mode)

INFO: [SYNCHK 200-10] 0 error(s), 1 warning(s).
INFO: [HLS 200-111] Finished Checking Synthesizability Time (s): cpu = 00:00:37 ; elapsed = 00:00:40 . Memory (MB): peak = 885.617 ; gain = 164.164 ; free physical = 1517 ; free virtual = 9701
INFO: [XFORM 203-1101] Packing variable 'out_pack.data.V' (firmware/nnet_utils/nnet_activation_stream.h:170) into a 48-bit variable.
INFO: [XFORM 203-1101] Packing variable 'out_data.data.V' (firmware/nnet_utils/nnet_activation_stream.h:64) into a 48-bit variable.
INFO: [XFORM 203-1101] Packing variable 'out_data.data.V' (firmware/nnet_utils/nnet_activation_stream.h:64) into a 30-bit variable.
INFO: [XFORM 203-1101] Packing variable 'out_data.data.V' (firmware/nnet_utils/nnet_activation_stream.h:64) into a 96-bit variable.
INFO: [XFORM 203-1101] Packing variable 'out_data.data.V' (firmware/nnet_utils/nnet_activation_stream.h:64) into a 72-bit variable.
INFO: [XFORM 203-1101] Packing variable 'out_data.data.V' (firmware/nnet_utils/nnet_activation_str

...

INFO: [Timing 38-35] Done setting XDC timing constraints.
INFO: [Pwropt 34-9] Applying IDT optimizations ...
INFO: [Pwropt 34-10] Applying ODC optimizations ...
INFO: [Physopt 32-619] Estimated Timing Summary | WNS=2.231 | TNS=0.000 |
Running Vector-less Activity Propagation...

Finished Running Vector-less Activity Propagation


Starting PowerOpt Patch Enables Task
INFO: [Pwropt 34-162] WRITE_MODE attribute of 0 BRAM(s) out of a total of 188 has been updated to save power. Run report_power_opt to get a complete listing of the BRAMs updated.
INFO: [Pwropt 34-201] Structural ODC has moved 178 WE to EN ports
Number of BRAM Ports augmented: 7 newly gated: 178 Total Ports: 376
Number of Flops added for Enable Generation: 4

Ending PowerOpt Patch Enables Task | Checksum: 10db46cec

Time (s): cpu = 00:00:00.76 ; elapsed = 00:00:00.75 . Memory (MB): peak = 3565.469 ; gain = 0.000 ; free physical = 4293 ; free virtual = 7935
Ending Power Optimization Task | Checksum: 10db46cec

Time (s): cpu =

Phase 2.2.1 Physical Synthesis In Placer | Checksum: aeacfda0

Time (s): cpu = 00:02:26 ; elapsed = 00:01:00 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 4072 ; free virtual = 7671
Phase 2.2 Global Placement Core | Checksum: 1b3831e38

Time (s): cpu = 00:02:32 ; elapsed = 00:01:02 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 4065 ; free virtual = 7659
Phase 2 Global Placement | Checksum: 1b3831e38

Time (s): cpu = 00:02:32 ; elapsed = 00:01:02 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 4108 ; free virtual = 7702

Phase 3 Detail Placement

Phase 3.1 Commit Multi Column Macros
Phase 3.1 Commit Multi Column Macros | Checksum: 199f8f6c2

Time (s): cpu = 00:02:42 ; elapsed = 00:01:05 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 4081 ; free virtual = 7688

Phase 3.2 Commit Most Macros & LUTRAMs
Phase 3.2 Commit Most Macros & LUTRAMs | Checksum: 106317ca7

Time (s): cpu = 00:03:01 ; elapsed = 00:01:12 . Memory (MB):

report_control_sets: Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 4100 ; free virtual = 7786
Command: route_design
Attempting to get a license for feature 'Implementation' and/or device 'xc7z020'
INFO: [Common 17-349] Got license for feature 'Implementation' and/or device 'xc7z020'
Running DRC as a precondition to command route_design
INFO: [DRC 23-27] Running DRC with 8 threads
INFO: [Vivado_Tcl 4-198] DRC finished with 0 Errors
INFO: [Vivado_Tcl 4-199] Please refer to the DRC report (report_drc) for more information.


Starting Routing Task
INFO: [Route 35-254] Multithreading enabled for route_design using a maximum of 8 CPUs
Checksum: PlaceDB: ad1b4c18 ConstDB: 0 ShapeSum: ba7b05cf RouteDB: 0

Phase 1 Build RT Design
Phase 1 Build RT Design | Checksum: fa1e3061

Time (s): cpu = 00:00:26 ; elapsed = 00:00:14 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 3962 ; free virtual = 7649
Post Restoration Check

Writing XDEF routing.
Writing XDEF routing logical nets.
Writing XDEF routing special nets.
Write XDEF Complete: Time (s): cpu = 00:00:12 ; elapsed = 00:00:04 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 3488 ; free virtual = 7416
INFO: [Common 17-1381] The checkpoint '/home/nexuscore/EGYETEM/DIPLOMAMUNKA/DATA_ACQUISITION/TUDATALLAPOT_QUANTIZED_VER0_312_FULL/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_routed.dcp' has been generated.
write_checkpoint: Time (s): cpu = 00:00:24 ; elapsed = 00:00:15 . Memory (MB): peak = 3629.531 ; gain = 0.000 ; free physical = 3627 ; free virtual = 7431
INFO: [runtcl-4] Executing : report_drc -file design_1_wrapper_drc_routed.rpt -pb design_1_wrapper_drc_routed.pb -rpx design_1_wrapper_drc_routed.rpx
Command: report_drc -file design_1_wrapper_drc_routed.rpt -pb design_1_wrapper_drc_routed.pb -rpx design_1_wrapper_drc_routed.rpx
INFO: [IP_Flow 19-1839] IP Catalog is up to date.
INFO: [DRC 23-27] Running DRC with 

INFO: [Designutils 20-2272] Running write_bitstream with 8 threads.
Loading data files...
Loading site data...
Loading route data...
Processing options...
Creating bitmap...
Creating bitstream...
Writing bitstream ./design_1_wrapper.bit...
INFO: [Vivado 12-1842] Bitgen Completed Successfully.
INFO: [Project 1-120] WebTalk data collection is mandatory when using a WebPACK part without a full Vivado license. To see the specific WebTalk data collected for your design, open the usage_statistics_webtalk.html or usage_statistics_webtalk.xml file in the implementation directory.
INFO: [Common 17-186] '/home/nexuscore/EGYETEM/DIPLOMAMUNKA/DATA_ACQUISITION/TUDATALLAPOT_QUANTIZED_VER0_312_FULL/myproject_vivado_accelerator/project_1.runs/impl_1/usage_statistics_webtalk.xml' has been successfully sent to Xilinx on Sun Nov 14 10:42:48 2021. For additional details about this file, please refer to the WebTalk help file at /tools/Xilinx/Vivado/2019.1/doc/webtalk_introduction.html.
INFO: [Common 17-83]